In [1]:
import yaml
import json
import pandas as pd
from mlflow import start_run, log_metric, log_param, set_experiment
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with open("params.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [2]:
train_labels = pd.read_csv(config["train_y_dataset_path"])
train_predictions = pd.read_csv(config["train_predictions_path"])

In [3]:
validation_labels = pd.read_csv(config["validation_y_dataset_path"])
validation_predictions = pd.read_csv(config["validation_predictions_path"])

In [4]:
def log_to_mlflow(experiment_name, run_name, metric_names, metric_results):
    set_experiment(experiment_name)
    with start_run(run_name=run_name):
        [log_metric(name, result) for name, result in zip(metric_names, metric_results)]

In [5]:
def log_to_markdown(file_path, experiment_name, run_name, metric_names, metric_results):
    with open(file_path, 'w') as file:
        file.write(f'**{experiment_name} - {run_name}**\n\n')
        [file.write(f'| {name} ') for name in metric_names]
        file.write('|\n')
        file.write('| ----- '*len(metric_names))
        file.write('|\n')
        [file.write(f'| {round(result, 6)} ') for result in metric_results]
        file.write('|\n')

In [6]:
metric_names = [
    "train accuracy",
    "train recall",
    "train precision",
    "train f1_score",
    
    "validation_accuracy",
    "validation recall",
    "validation_precision",
    "validation f1_score"
]

metric_results = [
    accuracy_score(train_labels, train_predictions),
    recall_score(train_labels, train_predictions),
    precision_score(train_labels, train_predictions),
    f1_score(train_labels, train_predictions),
    
    accuracy_score(validation_labels, validation_predictions),
    recall_score(validation_labels, validation_predictions),
    precision_score(validation_labels, validation_predictions),
    f1_score(validation_labels, validation_predictions)
]

In [7]:
log_to_mlflow(config["experiment_name"], config["run_name"], metric_names, metric_results)
log_to_markdown(config["metrics_path"], config["experiment_name"], config["run_name"], metric_names, metric_results)